In [12]:
# Libraries
import os
import json
import pandas as pd
import numpy as np
from PIL import Image
from glob import glob

import matplotlib.pyplot as plt
import matplotlib.patches as patches


import warnings
warnings.filterwarnings('ignore')

In [13]:
# mmdetection에 입력가능한 coco data format을 만들기 위해 폴더안에 있는 파일들을 리스트화 한다.
# train / val / final
# original train data는 전체 4000개이지만 본 코드에서는 이미 이상치를 삭제하여 3996개로 프린트 될 수 있음.

files = glob('./data/anno_train/*')
len(files)

3996

In [14]:
# convert annotations to coco format
# 'images' key에 있는 id와 'annotations' key에 있는 image_id에 순차적으로 index 부여
# 'annotaions' key에 있는 id에는 cnt라는 counter operator를 이용하여 순차적으로 index 부여

imgs = []
ann = []
err_id = []
cnt = 0    # id counter for annotations

for idx, fl in enumerate(files, 1):
    # error 발생하는 파일 찾아서 삭제
    try:
        dir = open(fl)
        info = json.load(dir)
    except:
        print(fl)
        continue

    img = info['images'][0]
    img['id'] = idx
    
    temp = []
    for j in range(len(info['annotations'])):
        cnt += 1
        a = info['annotations'][j]
        a['image_id'] = idx
        a['id'] = cnt
        temp.append(a)

    cat = info['categories']
    imgs.append(img)
    ann.append(temp)
       
# transform 2D list to 1D list
ann = sum(ann, [])        
    
dataset = {}

dataset["images"] = imgs
dataset["categories"] = cat 
dataset["annotations"] = ann

# save dataset as json format
with open('./data/train.json', 'w') as f:
    json.dump(dataset, f)

In [15]:
# It happens to be so that the bounding boxes are lists and not ndarrays? Ndarrays are not JSON serialisable
# TypeError: Argument 'bb' has incorrect type (expected numpy.ndarray, got list)
# annotation이 polygon 형태가 아니라 pixel이나 line으로 된 경우 에러 발생
# 잘못된 annotation을 제거하기 위해 image_id searching

# Find JSON that gives errors
JSON_LOC="/home/kerrykim/PycharmProjects/6.plastic_segmentation/mmdetection/data/train.json"

#Open JSON
val_json = open(JSON_LOC, "r")
json_object = json.load(val_json)
val_json.close()

for i, instance in enumerate(json_object["annotations"]):
    if len(instance["segmentation"][0]) < 8:
        print("instance number", i, "raises arror:", instance["segmentation"][0], 
              instance["image_id"]) 

In [16]:
# image_id = 111, 3354로 파일이름을 찾아냄
# print(dataset['images'][110])
# print(dataset['images'][3352])

In [17]:
# PS_038_4646.json, PS_042_7482.json, PS_038_2948.jpg, PET_015_1358.jpg 파일에 대하여 train/annotation 폴더에서 삭제